# Eat Safe, Love

## Notebook Set Up

In [67]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [68]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [69]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [70]:
# review the collections in our database
collections = db.list_collection_names()
pprint(collections)


['establishments']


In [71]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [72]:
import pprint

In [73]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f'Number of documents with a hygiene score of 20: {count}')
# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint.pprint(first_document)


Number of documents with a hygiene score of 20: 41
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': '0',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6518c69fff6ccd54d904f2ff'),
 'geocode': {'latitude': 50, 'longitude': 0},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}],
 'meta': {'d

In [74]:
# Convert the result to a Pandas DataFrame
import pandas as pd

# Display the number of rows in the DataFrame
df = pd.DataFrame(list(establishments.find(query)))

# Display the first 10 rows of the DataFrame
print(f'Number of rows in DataFrame: {len(df)}')
print(df.head(10))

Number of rows in DataFrame: 41
                        _id   FHRSID  ChangesByServerID   
0  6518c69fff6ccd54d904f2ff   110681                  0  \
1  6518c69fff6ccd54d904f67a   612039                  0   
2  6518c69fff6ccd54d904f988   730933                  0   
3  6518c69fff6ccd54d904fb70   172735                  0   
4  6518c69fff6ccd54d904fb7e   172953                  0   
5  6518c69fff6ccd54d9050528   512854                  0   
6  6518c69fff6ccd54d9050740  1537089                  0   
7  6518c69fff6ccd54d9051c67   155648                  0   
8  6518c69fff6ccd54d90520ae  1012883                  0   
9  6518c69fff6ccd54d90528bf   644109                  0   

  LocalAuthorityBusinessID               BusinessName   
0                     4029        The Chase Rest Home  \
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza   
4             PI/000024532       

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [75]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
  "local_authority.LocalAuthorityName": { "$regex": "london", "$options": "i" },
  "RatingValue": { "$gte": 4 }
}
# Use count_documents to display the number of documents in the result
count = db.establishments.count_documents(query)
print(f'Number of documents in \'establishments\': {count}')
# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint.pprint(first_document)


Number of documents in 'establishments': 0
None


In [76]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(establishments.find(query)))
# Display the number of rows in the DataFrame
print(f'Number of rows in DataFrame: {len(df)}')
# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows in DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [83]:
# Search within 0.01 degree on either side of the latitude and longitude.

degree_search = 0.01
penang_flavours = establishments.find_one({"BusinessName": "Penang Flavours"})
latitude = float(penang_flavours["geocode"]["latitude"])
longitude = float(penang_flavours["geocode"]["longitude"])
# Rating value must equal 5
query = {
    "RatingValue": 5,
    "geocode.latitude": {
        "$gte": latitude - degree_search,
        "$lte": latitude + degree_search
    },
    "geocode.longitude": {
        "$gte": longitude - degree_search,
        "$lte": longitude + degree_search
    }
}

# Sort by hygiene score
sort = [("scores.Hygiene", 1)]

# Print the results
top_5_establishments = establishments.find(query).sort(sort).limit(5)
for establishment in top_5_establishments:
    pprint.pprint(establishment)

In [84]:
# Convert result to Pandas DataFrame
import pandas as pd
establishments_list = list(top_5_establishments)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [94]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    {
        "$match": {
            "scores.Hygiene": 0
        }
    },
    {
        "$group": {
            "_id": "$local_authority.LocalAuthorityName",
            "count": {"$sum": 1}
        }
    },
    {
        "$sort": {
            "count": -1
        }
    }
]
result = establishments.aggregate(pipeline)

# Print the number of documents in the result
count = 0
# Display the first 10 results using pprint
for document in result:
    if count < 10:
        pprint.pprint(document)
        count += 1
    else:
        break
print(f"Number of documents: {count}")

{'_id': None, 'count': 16827}
Number of documents: 1


In [95]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame


NameError: name 'result_list' is not defined